In [1]:
import sagemaker.huggingface

In [2]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::707684582322:role/service-role/AmazonSageMaker-ExecutionRole-20191024T163188
sagemaker bucket: sagemaker-eu-west-1-707684582322
sagemaker session region: eu-west-1


In [3]:
import boto3
import os

prefix = 'DEMO-huggingface-finetune-twitter'
s3_input_train_validation = 's3://{}/{}/train'.format(sagemaker_session_bucket, prefix)

boto3.Session().resource('s3').Bucket(sagemaker_session_bucket).Object(os.path.join(prefix, 'train/sem_eval_2018_task_1_train.csv')).upload_file('./dataset/sem_eval_2018_task_1_train.csv')
boto3.Session().resource('s3').Bucket(sagemaker_session_bucket).Object(os.path.join(prefix, 'train/sem_eval_2018_task_1_validation.csv')).upload_file('./dataset/sem_eval_2018_task_1_validation.csv')

s3_input_train_validation

's3://sagemaker-eu-west-1-707684582322/DEMO-huggingface-finetune-twitter/train'

In [4]:
!pygmentize ./scripts/train_nlp_bert_sm_compatible.py 

import pandas as pd 
import numpy as np
import torch 
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel, BertConfig

import torch.nn as nn
import torch.nn.functional as F

import argparse
import logging
import os
import random
import sys

from datetime import datetime

if 'SM_MODEL_DIR' in os.environ:
    pass
else:
    os.environ["SM_MODEL_DIR"] = "./"
    os.environ["SM_CHANNEL_TRAIN"] = "./"


class MyNLPDataset(Dataset):
    def __init__(self, file_name, model_name):
        # data loading
        df = pd.read_csv(file_name)
        self.data_en = df.loc[:,["Tweet"]].squeeze().values.tolist()
        self.label_index = torch.from_numpy(df.iloc[:,2:].astype(int).to_numpy())
        self.n_samples = df.shape[0]
        self.embedding_model = None
        self.tokenizer = None
        self.model_name = model_name
        self.tokenized_text = [ {} for i in range(df.shape[0])]
        #self.token_type_ids = None
        #se

In [14]:
from sagemaker.huggingface import HuggingFace
import time

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 100,                          # number of training epochs
                 'train_batch_size': 64,               # batch size for training
                 'learning_rate': 0.000001,                # learning rate used during training
                 'model_id': "bert-base-multilingual-uncased", # pre-trained model
                }

In [15]:
# define Training Job Name 
job_name = f'huggingface-finetune-twitter{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train_nlp_bert_sm_compatible.py', # fine-tuning script used in training jon
    source_dir           = './scripts',       # directory where fine-tuning script is stored
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    #instance_type        = 'local_gpu',   # instances type used for the training job    
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.6',           # the transformers version used in the training job
    pytorch_version      = '1.7',           # the pytorch_version version used in the training job
    py_version           = 'py36',            # the python version used in the training job
    hyperparameters      = hyperparameters,   # the hyperparameter used for running the training job
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [16]:
# define a data input dictonary with our uploaded s3 uris

data = {
    #'train': train_local_location,
    'train': s3_input_train_validation,
}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=False)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-finetune-twitter2023-03-16--2023-03-16-11-34-44-518
